In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
PATH_TO_MEDS_CSV = # TODO path to manually cleaned treatment file
meds_df = pd.read_csv(PATH_TO_MEDS_CSV)
processed_datadir = # TODO path to processed data using other scripts
pd_totals_df = pd.read_csv(processed_datadir + 'PD_totals_across_time.csv')
pd_baseline_df = pd.read_csv(processed_datadir + 'PD_baseline.csv')
pd_other_df = pd.read_csv(processed_datadir + 'PD_other_across_time.csv')

In [ ]:
# record first line treatments that are levodopa or rasigiline
meds_df['STARTDT_date'] = pd.to_datetime(meds_df['STARTDT'])
meds_df.sort_values(by='STARTDT_date', inplace=True)
first_line_meds_df = meds_df.drop_duplicates(subset='PATNO', keep='first')
included_trts = {'LEVODOPA','RASAGILINE'}
first_line_meds_df = first_line_meds_df.loc[first_line_meds_df['CMTRT'].isin(included_trts)]
# 1 for levodopa, 0 for rasagiline
first_line_meds_df['treatment'] = np.where(first_line_meds_df['treatment']=='LEVODOPA', 1, 0)

In [ ]:
# take the maximum MDS-UPDRS part III score that was measured at a visit
pd_totals_df['NUPDRS3_max'] = pd_totals_df[['NUPDRS3_untreated','NUPDRS3_on',
                                            'NUPDRS3_off','NUPDRS3_maob']].max(skipna=True, axis=1)
# compute total part II and III score for motor assessment
pd_totals_df['mds23'] = pd_totals_df[['NUPDRS2','NUPDRS3_max']].sum(axis=1)
# merge in age
pd_other_df = pd_other_df[['PATNO','INFODT','AGE']].rename(columns={'AGE':'age'})
pd_totals_df = pd_totals_df.merge(pd_other_df, on=['PATNO','INFODT'])
# record last measurement before treatment start
pd_totals_df['INFODT_date'] = pd.to_datetime(pd_totals_df['INFODT'])
pd_totals_df = pd_totals_df.rename(columns = {'INFODT_DIS_DUR': 'disdur'})
df = first_line_meds_df['PATNO','treatment','STARTDT_date'].merge(pd_totals_df[['PATNO','mds23','INFODT_date',
                                                                                'disdur']],
                                                                  on = 'PATNO')
df = df.loc[df['INFODT_date'] <= df['STARTDT_date']]
df = df.sort_values(by='INFODT_date')
df = df.drop_duplicates(subset=['PATNO'], keep='last')

# merge in site number
pd_baseline_df = pd_baseline_df[['PATNO','CNO']].rename(columns={'CNO':'site'})
df = df.merge(pd_baseline_df, on='PATNO')

# save data
del df['PATNO']
del df['INFODT_date']
del df['STARTDT_date']
df.to_csv('line1_levodopa_rasagiline_age_mds23_disdur.csv',index=False)